In [ ]:
!pip install --upgrade plotly

In [ ]:
import locale
print(locale.getlocale())


('en_US', 'UTF-8')


In [ ]:
!export LANGUAGE=es_CL.UTF-8
!export LANG=es_CL.UTF-8
!export LC_ALL=es_CL.UTF-8
!locale-gen es_CL.UTF-8

Generating locales (this might take a while)...
  es_CL.UTF-8... done
Generation complete.


In [ ]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import os


In [ ]:
# Load the Drive helper and mount
#from google.colab import drive

# This will prompt for authorization.
#drive.mount('/content/drive')

#FOLDER_PATH='/content/drive/MyDrive/Proyecto AI/economicos/data_cruda'
#FILES = [x for x in os.listdir(FOLDER_PATH) if "json" in x ]

In [ ]:
FOLDER_PATH='/content/drive/MyDrive/Master/Taller AI/Proyecto AI/economicos/data_cruda'

In [ ]:
FILES = [x for x in os.listdir(FOLDER_PATH) if "json" in x ]

data_cruda = pd.concat([pd.read_json(f"{FOLDER_PATH}/{fpath}") for fpath in FILES]).reset_index().drop_duplicates(ignore_index=True, subset=['url']).drop('index', axis=1)

data_cruda.shape

(662262, 16)

In [ ]:
data_cruda = data_cruda.rename(columns = {
    'url': 'url',
    'description': 'description',
    'price': 'price',
    'title': 'title',
    'address': 'address',
    'images': 'images',
    'Tipo:': 'type',
    'Operación:': 'operation',
    'm  construidos:': 'm_built',
    'm  terreno:': 'm_size',
    'Región:': 'state',
    'Comuna:': 'county',
    'Fecha Publicación:': 'date',
    'Diario:': 'source',
    'Dormitorios:':'rooms',
    'Baños:': 'bathrooms'
})

In [ ]:
data_cruda.count()

url            662262
description    662262
price          662262
title          662259
address        662257
images         662262
type           650760
operation      662138
m_built        430817
m_size         358809
state          644703
county         631854
date           662261
source         181921
rooms          424232
bathrooms      409015
dtype: int64

In [ ]:
data_cruda.select_dtypes(include=['float', 'int']).describe().T

,count,mean,std,min,25%,50%,75%,max
m_built,430817.0,2.869462e+10,1.701334e+13,0.0,50.0,76.0,136.0,1.111111e+16
m_size,358809.0,2.878534e+11,1.669768e+14,-1000.0,61.0,145.0,540.0,1.000000e+17
rooms,424232.0,3.998401e+04,6.315758e+06,1.0,2.0,3.0,3.0,9.982525e+08
bathrooms,409015.0,1.890982e+00,1.307144e+00,1.0,1.0,2.0,2.0,2.800000e+02


In [ ]:
data_cruda.select_dtypes(include=['object']).columns

Index(['url', 'description', 'price', 'title', 'address', 'images', 'type',
       'operation', 'state', 'county', 'date', 'source'],
      dtype='object')

In [ ]:
data_cruda[['type', 'operation', 'state', 'county']].describe().T

,count,unique,top,freq
type,650760,25,Departamento,256025
operation,662138,5,Venta,335240
state,644703,17,Metropolitana de Santiago,327387
county,631854,424,Santiago,79880


In [ ]:
import plotly.express as px
cols = ["state"]
title = "Regiones"
data = data_cruda.groupby(cols)[[cols[-1]]].count().rename(columns={cols[-1]:'count'}).reset_index()
fig = px.treemap(data, path=[px.Constant(title)] + cols, values='count', width=300, height=500, branchvalues ='total')
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.data[0].textinfo = "label+value+percent parent"
fig.show()

In [ ]:
import plotly.express as px
cols = ["type"]
title = "Tipo de Propiedad"
data = data_cruda.groupby(cols)[[cols[-1]]].count().rename(columns={cols[-1]:'count'}).reset_index()
fig = px.treemap(data, path=[px.Constant(title)] + cols, values='count', width=300, height=500, branchvalues ='total')
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.data[0].textinfo = "label+value+percent parent"
fig.show()

In [ ]:
import plotly.express as px

#& (data_cruda.operation.isin(['Venta', 'Arriendo']) )
arriendo = ['Arriendo']
venta = ['Venta']
va = arriendo + venta
boo = data_cruda.state.isin(['Metropolitana de Santiago'])  
d1 = data_cruda[boo]
boo &= data_cruda['type'].isin(['Departamento', 'Departamento Amoblado', 'Casa'])
d2 = d1[boo]

var = 'price'
boo &= ~data_cruda[var].isna()
d3 = d2[boo]

var = 'rooms'
boo &= ~data_cruda[var].isna()
d4 = d3[boo]

var = 'bathrooms'
boo &= ~data_cruda[var].isna()
d5 = d4[boo]

var = 'm_size'
boo &= ~data_cruda[var].isna()
d6 = d5[boo]

var = 'm_built'
boo &= ~data_cruda[var].isna()
d7 = d6[boo]


stages = ["Total", "Metropolitana de Santiago", "Departamento & Casa","Info precio",'Info dorms','Info baños','Info size','Info built']

df_arriendo = pd.DataFrame(
    dict(
        number=[data_cruda[data_cruda.operation.isin(arriendo)].shape[0],
                d1[d1.operation.isin(arriendo)].shape[0],
                d2[d2.operation.isin(arriendo)].shape[0],
                d3[d3.operation.isin(arriendo)].shape[0],
                d4[d4.operation.isin(arriendo)].shape[0],
                d5[d5.operation.isin(arriendo)].shape[0],
                d6[d6.operation.isin(arriendo)].shape[0],
                d7[d7.operation.isin(arriendo)].shape[0]
                ],
        stages=stages,
        Tipo=["Arriendo"]*len(stages)
    )
)

df_venta = pd.DataFrame(
    dict(
        number=[data_cruda[data_cruda.operation.isin(venta)].shape[0],
                d1[d1.operation.isin(venta)].shape[0],
                d2[d2.operation.isin(venta)].shape[0],
                d3[d3.operation.isin(venta)].shape[0],
                d4[d4.operation.isin(venta)].shape[0],
                d5[d5.operation.isin(venta)].shape[0],
                d6[d6.operation.isin(venta)].shape[0],
                d7[d7.operation.isin(venta)].shape[0]
                ],
        stages=stages,
        Tipo=["Venta"]*len(stages)
    )
)
df_otro = pd.DataFrame(
    dict(
        number=[data_cruda[~data_cruda.operation.isin(va)].shape[0],
                d1[~d1.operation.isin(va)].shape[0],
                d2[~d2.operation.isin(va)].shape[0],
                d3[~d3.operation.isin(va)].shape[0],
                d4[~d4.operation.isin(va)].shape[0],
                d5[~d5.operation.isin(va)].shape[0],
                d6[~d6.operation.isin(va)].shape[0],
                d7[~d7.operation.isin(va)].shape[0]
                ],
        stages=stages,
        Tipo=["Otro"]*len(stages)
    )
)
data = pd.concat([df_arriendo, df_venta, df_otro], axis=0)

fig = px.funnel(data, y='number', x='stages', color="Tipo", title="Filtrado", width=500, height=500, orientation='v')
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
#fig.update_layout(texttemplate ="%{value:$,s}")
#fig.update_layout(showlegend=False)
fig.update_xaxes(tickformat = ',.0d', hoverformat = ',.0d')
fig.update_yaxes(tickformat = ',.0d', hoverformat = ',.0d')
fig.update_traces(marker_colorbar_tickformat = ',.0d')

fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [ ]:
import uuid
def uuid3(url):
  return str(uuid.uuid3(uuid.NAMESPACE_URL, url))
def uuid3array(arr):
  return [str(uuid.uuid3(uuid.NAMESPACE_URL, url)) for url in arr]
d7["folder"] = d7.url.map(uuid3)
d7["images_uuid3"] = d7.images.map(uuid3array)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
d7[["folder", "images_uuid3"]].iloc[0]

folder                       4dcfc6ce-b2ff-3c0e-8438-e89c9bc45207
images_uuid3    [d5127bec-1b9c-36ab-a21f-2a8db63555bf, 693a03a...
Name: 14, dtype: object

In [ ]:
d7[["url", "folder", "images_uuid3"]].to_csv("d7.csv")

In [ ]:
d7.images.map(len).describe()

count    141885.000000
mean          7.887684
std           4.663189
min           0.000000
25%           5.000000
50%           9.000000
75%          10.000000
max          62.000000
Name: images, dtype: float64

In [ ]:
d7.images.map(len).sum()

1119144

In [ ]:
d7.images_uuid3.map(len).sum()

1119144

In [ ]:
d7.images_uuid3.map(len).sum() - d7.images.explode().drop_duplicates().count()

7890

In [ ]:
d7.description.map(len).describe()

count    141885.000000
mean        631.267815
std         423.463368
min           1.000000
25%         325.000000
50%         561.000000
75%         845.000000
max       13754.000000
Name: description, dtype: float64

In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(d7.description.sample(10).iloc[0])

'CPROENCINA Nancy Auvelez Tobar 984713913 cproencina@gmail.com  Se vende antiguo departamento en la Comuna de Peñalolén Cercano a todo tipo de comercio, colegios, supermercados. Se encuentra en remodelación, ventanas termopanel y fachada del condominio\n  Cuenta con un estacionamiento asignado al departamento Edificio de cuatro pisos, se ubica en el segundo piso, sin ascensor Con Orientación al Oriente Muy luminoso, vista despejada\n  Detalles del departamento : Consta de tres dormitorios,  un baño  Living Comedor  Cocina separada ,  instalación de lavadora ,  El sistema de gas es cilindro   Edificio cuenta con Puertas electrónicas ,  áreas verdes   no paga contribuciones ,  bajos gastos comunes,   Coordine su visita !!!'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import plotly.express as px

#& (data_cruda.operation.isin(['Venta', 'Arriendo']) )
arriendo = ['Arriendo']
venta = ['Venta']
va = arriendo + venta
d1 = data_cruda[(data_cruda.state.isin(['Metropolitana de Santiago']) ) ]
d2 = data_cruda[(data_cruda.state.isin(['Metropolitana de Santiago']) ) & (data_cruda['type'].isin(['Departamento', 'Departamento Amoblado', 'Casa']))]

stages = ["Total", "Metropolitana de Santiago", "Departamento & Casa"]

df_arriendo = pd.DataFrame(
    dict(
        number=[data_cruda[data_cruda.operation.isin(arriendo)].shape[0], d1[d1.operation.isin(arriendo)].shape[0], d2[d1.operation.isin(arriendo)].shape[0]],
        stages=stages,
        Tipo=["Arriendo"]*len(stages)
    )
)

df_venta = pd.DataFrame(
    dict(
        number=[data_cruda[data_cruda.operation.isin(venta)].shape[0], d1[d1.operation.isin(venta)].shape[0], d2[d1.operation.isin(venta)].shape[0]],
        stages=stages,
         Tipo=["Venta"]*len(stages)
    )
)
df_otro = pd.DataFrame(
    dict(
        number=[data_cruda[~data_cruda.operation.isin(va)].shape[0], d1[~d1.operation.isin(va)].shape[0], d2[~d1.operation.isin(va)].shape[0]],
        stages=stages,
         Tipo=["Otro"]*len(stages)
    )
)

data = pd.concat([df_arriendo, df_venta, df_otro], axis=0)

fig = px.funnel(data, y='number', x='stages', color="Tipo", title="Filtrado", width=300, height=500, orientation='v')
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
#fig.update_layout(showlegend=False)

fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [ ]:
data_cruda.columns

Index(['url', 'description', 'price', 'title', 'address', 'images', 'type',
       'operation', 'm_built', 'm_size', 'state', 'county', 'date', 'source',
       'rooms', 'bathrooms'],
      dtype='object')

In [ ]:
import plotly.express as px
cols = ["operation", "type"]
title = "Data Seleccionada: Metropolitana de Santiago"
# , 'Valparaíso'
selected_data = data_cruda[(data_cruda.state.isin(['Metropolitana de Santiago']) ) & (data_cruda.operation.isin(['Venta', 'Arriendo']) ) & (data_cruda['type'].isin(['Departamento', 'Departamento Amoblado', 'Casa']))]
data = selected_data.groupby(cols)[[cols[-1]]].count().rename(columns={cols[-1]:'count'}).reset_index()
fig = px.treemap(data, path=[px.Constant(title)] + cols, values='count', width=300, height=500, branchvalues ='total')
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.data[0].textinfo = "label+value"
fig.show()

In [ ]:
selected_data.count()

url            240448
description    240448
price          240448
title          240446
address        240446
images         240448
type           240448
operation      240448
m_built        192933
m_size         148597
state          240448
county         240448
date           240448
source          67459
rooms          227194
bathrooms      215238
dtype: int64

In [ ]:
selected_data[selected_data.state == 'Metropolitana de Santiago'].count()

url            240448
description    240448
price          240448
title          240446
address        240446
images         240448
type           240448
operation      240448
m_built        192933
m_size         148597
state          240448
county         240448
date           240448
source          67459
rooms          227194
bathrooms      215238
dtype: int64

In [ ]:
import plotly.express as px
cols = ["operation"]
title = "Operación"
data = data_cruda.groupby(cols)[[cols[-1]]].count().rename(columns={cols[-1]:'count'}).reset_index()
fig = px.treemap(data, path=[px.Constant(title)] + cols, values='count', width=300, height=500, branchvalues ='total')
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.data[0].textinfo = "label+value+percent parent"
fig.show()

In [ ]:
import re

def money(amount):
  new_amount = re.sub(',','.', re.sub(r'[\s$\.UFSD]', '', amount)).strip()
  if new_amount == '':
    return 0
  new_amount = float(new_amount)
  if 'UF' in amount:
    return new_amount * 29000
  elif 'USD' in amount:
    return new_amount * 800
  else:
    return new_amount
data_cruda['new_price'] = data_cruda.price.map(money, na_action='ignore')

In [ ]:
[d.url.count() for d in [d1, d2, d3, d4, d5, d6, d7]]

[327387, 242989, 242989, 229069, 215070, 144365, 141885]

In [ ]:
col = "type"
data = selected_data.groupby(['state', col])[[col]].count().rename(columns={col:'count'}).reset_index()
fig = px.bar(data[data['count']>500], x='count', y=col, color="state", orientation='h')
fig.update_layout( go.Layout( template='ggplot2', margin={'t':3, 'b':3, 'l':3, 'r':3}, yaxis={'categoryorder':'total ascending'} ))
fig.show()


In [ ]:
selected_data = d7[d7.operation.isin(va)]
selected_data.url.count()

141494

In [ ]:
dd =  pd.to_datetime(selected_data.date, infer_datetime_format=True)

In [ ]:
display(dd.max())
display(dd.min())

Timestamp('2021-10-09 11:09:00')

Timestamp('2018-02-18 17:41:49')

In [ ]:
selected_data.date.min()

'2018-02-18 17:41:49'

In [ ]:
selected_data.count()

url             141494
description     141494
price           141494
title           141494
address         141492
images          141494
type            141494
operation       141494
m_built         141494
m_size          141494
state           141494
county          141494
date            141494
source           21260
rooms           141494
bathrooms       141494
folder          141494
images_uuid3    141494
dtype: int64

In [ ]:
col = "bathrooms"
data = selected_data.groupby(['operation', col])[[col]].count().rename(columns={col:'count'}).reset_index()
fig = px.bar(data[data['count']>100], y='count', x=col, color='operation', orientation='v', width=500, height=200)
fig.update_layout( go.Layout( template='ggplot2', margin={'t':3, 'b':3, 'l':3, 'r':3}, yaxis={'categoryorder':'total ascending'} ))
#fig.update_layout(showlegend=False)
fig.show()

In [ ]:
col = "rooms"
data = selected_data.groupby(['operation', col])[[col]].count().rename(columns={col:'count'}).reset_index()
fig = px.bar(data[data['count']>100], y='count', x=col, color='operation', orientation='v', width=500, height=200)
fig.update_layout( go.Layout( template='ggplot2', margin={'t':3, 'b':3, 'l':3, 'r':3}, yaxis={'categoryorder':'total ascending'} ))
#fig.update_layout(showlegend=False)
fig.show()

In [ ]:
col = "m_size"
data = selected_data.groupby(['state', col])[[col]].count().rename(columns={col:'count'}).reset_index()
fig = px.bar(data[data['count']>1000], y='count', x=col, color="state", orientation='v', width=500, height=200)
fig.update_layout( go.Layout( template='ggplot2', margin={'t':3, 'b':3, 'l':3, 'r':3}, yaxis={'categoryorder':'total ascending'} ))
fig.show()

In [ ]:
col = "m_built"
data = selected_data.groupby(['state', col])[[col]].count().rename(columns={col:'count'}).reset_index()
fig = px.bar(data[data['count']>1000], y='count', x=col, color="state", orientation='v', width=500, height=200)
fig.update_layout( go.Layout( template='ggplot2', margin={'t':3, 'b':3, 'l':3, 'r':3}, yaxis={'categoryorder':'total ascending'} ))
fig.show()

In [ ]:
import plotly.express as px
col = "m_built"
data = d7[(d7[col] < 500) & (d7.operation.isin(va))].groupby(['operation', col])[[col]].count().rename(columns={col:'count'}).reset_index()
fig = px.box(data, y=col, color='operation', width=500, height=400, title='Metros Construidos')
#fig.update_layout(showlegend=False)
fig.show()

KeyError: ignored

In [ ]:
#regiones= ['Los Lagos', 'Los Ríos']
#regiones= ['Los Ríos']
regiones= ['Maule']
columnas = ['new_price', 'date', 'description', 'url', 'm_built', 'm_size']
with pd.option_context('display.max_colwidth', None, 'display.float_format', '{:,.0f}'.format) :  
  display(data_cruda[(data_cruda.type.isin([ 'Sitio o Terreno', 'Parcela o Chacra'])) & (data_cruda.state.isin(regiones) ) & (data_cruda.new_price < 6e7) & (data_cruda.new_price > 1e7) & (data_cruda.m_size> 2e4) & (data_cruda.m_size< 9e4)].sort_values(["new_price", 'date'], ascending=[True, False])[columnas].sample(2))

,new_price,date,description,url,m_built,m_size
281796,"55,000,000",2019-12-01 00:54:50,"hermoso terreno con árboles frutales. la casa de 72m2 consta; 3 dormitorios, 2 baños, living, cocina comedor, luz y agua propia mas una noria y una pileta de 8 mts .al lado de la casa tiene una pieza de 21mt2 y un techo de 75 mts. para mayor información y fotos llamar al número de Ximena .+56944880764",https://www.economicos.cl/propiedades/vendo-terreno-con-casa-codAAPYVCI.html,72,"40,000"
197135,"50,000,000",2019-06-12 12:31:00,"se vende Parcela en sector punta de diamante, San Clemente, con rol propio y derechos de agua, próximo a urbanizar, invernadero, precio conversable",https://www.economicos.cl/propiedades/parcela-san-clemente-codAAOXWUA.html,70,"28,900"


In [ ]:
data_cruda.describe().T

,count,mean,std,min,25%,50%,75%,max
m_built,430817.0,2.869462e+10,1.701334e+13,0.0,50.0,76.0,136.0,1.111111e+16
m_size,358809.0,2.878534e+11,1.669768e+14,-1000.0,61.0,145.0,540.0,1.000000e+17
rooms,424232.0,3.998401e+04,6.315758e+06,1.0,2.0,3.0,3.0,9.982525e+08
bathrooms,409015.0,1.890982e+00,1.307144e+00,1.0,1.0,2.0,2.0,2.800000e+02
new_price,662262.0,2.021096e+14,1.391226e+17,-200000.0,340000.0,2755000.0,95000000.0,1.111498e+20


In [ ]:
import re
import numpy as np
numbers = re.compile('[\.\,]?[0-9]*[0-9\.\,]*[0-9]+')
def com_number(number):
  n = number
  d = (x for x in (1e3, 1e4, 1e6, 1e9, 1e12, ))
  while n > 0.:
    yield n
    try:
      n = round(number / next(d), 1)
    except:
      n = 0

def c_to_set(number):
  return set(com_number(number))

def extract_numbers(text):
  for g in numbers.finditer(text):
    nn = g.group(0)
    if nn[0] in {',', '.',}:
      nn = "0" + nn
    nn = nn.replace('.', '').replace(",", ".").replace(".", "")
    yield float(nn)

def e_to_set(number):
  return set(extract_numbers(number))

def transformToComb(t):
  if isinstance(t, float):
    return set(com_number(t))
  else:
    return set(extract_numbers(t))
  
dd = data_cruda[~data_cruda.m_size.isnull()].reset_index(drop=True)
n = dd.m_size.iloc[0]
dd['union_desc'] = dd['description'] + " " + dd['title'].astype(str)
dd['s1'] = dd.m_size.map(transformToComb)
dd['s2'] = dd['union_desc'].map(transformToComb)
dd['s3'] = dd.m_built.map(transformToComb)
dd['intersection'] = dd['s1'].values & dd['s2'].values
dd['int2'] = dd['s3'].values & dd['s2'].values


In [ ]:
display(len(dd['s1']))
display(dd['s1'].values)
display(dd['s2'].values)
display(np.intersect1d(dd['s1'].values, dd['s2'].values))
display(dd['s1'].values & dd['s2'].values)

358809

array([{24.0}, {0.1, 90.0}, {0.1, 110.0}, ..., {0.1, 60.0}, {0.3, 290.0},
       {0.1, 51.0}], dtype=object)

array([{24.0, 85000.0, 2.0}, {1.0, 2.0, 3.0, 5.0, 10.0, 190000.0},
       {280000.0, 1.0, 2.0, 9.0, 65430684.0}, ...,
       {1.0, 2.0, 7.0, 9.0, 991077964.0, 530000.0, 90000.0, 24.0},
       {2.0, 290.0, 4.0, 9.0, 82122508.0, 56.0, 185.0, 6300.0},
       {130000000.0, 1.0, 6.0, 7.0, 991077964.0, 14.0, 95000.0, 24.0}],
      dtype=object)

array([set(), {5.0}, {5.0}, ..., {5000.0, 0.5, 5.0}, {1000.0, 1.0, 0.1},
       {0.1, 100.0}], dtype=object)

array([{24.0}, set(), set(), ..., set(), {290.0}, set()], dtype=object)

In [ ]:
dd.columns

Index(['url', 'description', 'price', 'title', 'address', 'images', 'type',
       'operation', 'm_built', 'm_size', 'state', 'county', 'date', 'source',
       'rooms', 'bathrooms', 'new_price', 'union_desc', 's1', 's2', 's3',
       'intersection', 'int2'],
      dtype='object')

In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(dd.loc[(dd['int2'] == set()) & ((dd['s1'] != set()) & (dd['s2'] != set())), ['union_desc', 'm_built',  's3', 's2', 'int2']].sample(3))
  

,union_desc,m_built,s3,s2,int2
84841,"Hermosa parcela con piscina ubicada en la localidad de el convento, comuna de Santo Domingo.\nConstrucción de 2 pisos con vista privilegiada, posee amplio living comedor, combustión lenta, cocina americana, equipada con gran luminosidad, cuenta con 4 dormitorios más 1 estar.\nAgua Potable \nGran Pozo\nLuz\nA 200 m de la carretera\nArboles frutales y ornmentales VENDO CASA CON PISCINA EN EL CONVENTO SANTO DOMINGO",NaN,{},"{200.0, 1.0, 2.0, 4.0}",{}
193332,INVIERTE EN CULTIVO DE CEREZAS! frutal de gran potencial y con crecimiento sostenido. Parcela con inmejorables condiciones para producir cerezos y criar caballares. Cuenta con 2.1 hectáreas de cerezo de 6 años en plena producción y acciones de agua.\n Cuenta con pesebreras cerradas techadas y pesebreras abiertas techadas. Actualmente posee un proyecto de riego tecnificado solar en proceso de aprobación. (CIR) Propiedad Agricola en Venta en Teno,NaN,{},"{21.0, 6.0}",{}
199213,"Parcelas ubicadas a unos 20 minutos aproximados de La Serena y unos 15 de Coquimbo, están sobre la carretera nueva a Ovalle. Condominio ya consolidado, sector de Quitallaco. EN VENTA SEGUNDA ETAPA. Alta plusvalía ya que están a solo unos 20 minutos de la ciudad de La Serena y están situadas sobre carretera. Frente al condominio se sitúa un paradero de la locomoción colectiva y a unos pocos metros el cruce peatonal. Todas las parcelas son planas, con 5000 m2 de superficie, luz, agua de pozo, reglamento de copropiedad y se entregan con Rol propio. INFORMACIÓN Y VISITAS con Marcela Meireles, Cel. +56987285834 - marcela.meireles@solutionhouse.cl Sitio o Terreno en Venta en Coquimbo",NaN,{},"{2.0, 5000.0, 56987285834.0, 15.0, 20.0}",{}


In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(dd.loc[(dd['int2'] != set())  & (dd["type"] == 'Departamento'), ['union_desc', 'm_built',  's3', 's2', 'int2']].head(10))
  

,union_desc,m_built,s3,s2,int2
4,"Se arrienda departamento en Santiago centro\n$ 330.000 (Trescientos treinta mil pesos)\n2 dormitorios, con closeth, 2 baños, living - comedor, cocina. Piso flotante. Vista Sur. Piso 11.\nSuperficie: 45 m2\nUbicado en General Jofré 107, Condominio Los Trinos.\nCercano al metro Bustamante, supermercado, bancos, centros médicos, colegios.\nContacto:\nCarmen Tobar P.\nCorredora de propiedades\nCelular: +56998378607\nFijo: 23249446 Depto 2 dormitorios 2 baños Santiago",45.0,{45.0},"{2.0, 23249446.0, 107.0, 11.0, 45.0, 56998378607.0, 330000.0}",{45.0}
24,"Valor 2.950 UF con derecho a estacionamiento descubierto para un vehículo. Orientación sur. Todos los dormitorios con clósets. Piso flotante living, comedor y pasillo, alfombra en dormitorios y cerámica en cocina. Cocina amoblada con cocina a gas natural. Un baño con tina, piso de cerámica. Departamento en segundo piso con amplia terraza. Departamento de 48 metros cuadrados. Condominio en barrio exclusivo de La Florida. Sector García Hurtado de Mendoza con Avenida Walker Martínez. Cuenta con conserjería las 24 horas. Condominio posee jardines, bancas de plaza, bicicletero, juegos infantiles, sala multiusos y piscina. A tres minutos del Mall Plaza Vespucio. Cerca de Metro Vicente Valdés. Gran conectividad con el resto de Santiago. Locomoción a la esquina. Cercano a Colegios, Universidades, Supermercados, Centros de Salud, Farmacias y Comercio en general. Llamar al +569 94839701 (Gestiona Dueño)\n Se Vende Departamento en Hermoso Condominio de La Florida",48.0,{48.0},"{2950.0, 48.0, 94839701.0, 24.0, 569.0}",{48.0}
42,"Bello Propiedades. ARRIENDA. Hermoso Departamento casi nuevo, en tranquilo sector residencial de Providencia, cercano a colegios supermercados, centros comerciales y metro Los Leones., ubicado en piso 9°, con orientación norte y mucha luz, consta de 85 m2 construidos y amplisima terraza de 21 m2 , con espectacular vista despejada a la cordillera, Cerro San Cristóbal y Costanera Center. Con , living comedor, cocina americana, logia, amplio dormitorio principal con walking closet y baño privado, amplio dormitorio secundario con amplios closets y baño privado. \n\nEs un departamento moderno y nuevo, equipado con calefacción (hay radiadores en todos los espacios), cocina totalmente equipada (encimeras, horno, cocina y campana totalmente nuevos), cocina con cubierta de cuarzo, ventanas de PVC con termopanel, piso de porcelanato en living, cocina y baños, piso fotolaminado en dormitorios y pasillo, iluminación led en todo el departamento, cortinas roller en todas la ventanas y alarma. Se encuentra en un exclusivo edificio con acceso biométrico, con conserjes 24/7, jardines, piscina, quincho y sala de eventos. El departamento cuenta con un estacionamiento y una bodega.\n \nInteresados en coordinar visita enviar correo a: contacto@bellopropiedades.cl o Whatsap: +569 7266 4823 Arriendo hermoso Depto.. Suecia - Los Leones",85.0,"{0.1, 85.0}","{2.0, 7266.0, 7.0, 9.0, 21.0, 85.0, 4823.0, 24.0, 569.0}",{85.0}
44,"En el centro de Quillota y a 3 cuadras , se encuentran DISPONIBLES 3 últimos departamentos.\nTendrás la comodidad de vivir cerca de todo...\nA pasos de Centro Médico ,colegios,bancos,supermercados, restaurantes y comercios.\nTransporte a la puerta.\nEl Edificio cuenta con locales comerciales y oficinas.\n\nDepartamentos Disponibles\n\n601\n37 m2\n1Dormitorio, 1 Baño \nTerraza\nWalking Closet\nCocina americana\nPiso 6, orientación sur\nValor UF 2.597 $ 75.000.000\n\n406\n50 m2\n2 Dormitorios, 2 Baños\nTerraza\nCocina Tradicional\nClosets en ambos dormitorios\nPiso 4, orientación Oriente\nEstacionamiento \nValor UF 3.749 $108.200.000\n\n305\n54 m2\n2 Dormitorios, 2 Baños\nTerraza\nCocina Tradicional\nClosets en ambos dormitorios\nPiso 3, orientación Norte\n2 Estacionamientos\nValor UF 4.046 $ 117.000.000\n\n\nLos departamentos cuentan con finas terminaciones y equipamiento de primer nivel que te sorprenderá\n

In [ ]:
dd['int2'].map(len).value_counts()

0    261723
1     96700
2       383
3         3
Name: int2, dtype: int64

In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(dd.loc[50260])

url                 https://www.economicos.cl/propiedades/arriendo-departamento-marzo-diciembre-codAAQK44A.html
description                                           3 dormitorios 2 baños cable wiffi gasto común incluidoa
price                                                                                                 $ 370.000
title                                                                     Arriendo departamento marzo diciembre
address                                                                  Linch con balmaceda Reñaca, Valparaíso
images                            [http://imgcf.ecn.cl/600/b6/b60746313b2fcf623137c7bcd3ee45e3664646ac.bin.jpg]
type                                                                                      Departamento Amoblado
operation                                                                                              Arriendo
m_built                                                                                                 